This creates a long file with clip FT magnitudes transposed and concatenated into a long series of samples

It's designed for a process that just grabs a window instead of using a prepared frame (since that creates massive data duplication)

In [4]:
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
import time
import cmath

In [8]:
# Get shared constants and functions
%run "NN Audio Core.py"

In [ ]:
# Number of all zero samples between clips
# TODO - not using pads at all, on assumption that the samples have a bit of buffer at start and end
#PAD_SIZE = 10


In [7]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Clean\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = get_ft(wav)
print(fft.shape)

(1025, 239)


In [34]:
# Creating data from clip wave files for adding to long data arrays

# Sample output is (samples, bins) all converted to magnitude
def get_samples(file):
    wav, rate = librosa.core.load(file)
    samples = abs(get_ft(wav).T) # organized as bins, frames so we need to transpose them to frames, bins
    return samples

In [35]:
def frames_file(data_path, max_samples):
    frames_file = data_path + "\\fsamples-" + str(max_samples)
    filename = os.fsdecode(frames_file)
    return filename

def targets_file(data_path, max_samples):
    targets_file = data_path + "\\ftargets-" + str(max_samples)
    filename = os.fsdecode(targets_file)
    return filename

# Iterate over clean & noisy folders to create frames and targets
# Updated to select clips based on difference between clean & noisy versions
def create_data(wav_root, data_path, max_samples = 10000, min_diff = 70, max_diff=300):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    sample_index = 0
    frames = np.memmap(frames_file(data_path, max_samples), mode='w+', dtype=DTYPE, shape=(max_samples,FFT_BINS,1))
    targets = np.memmap(targets_file(data_path, max_samples), mode='w+', dtype=DTYPE, shape=(max_samples,FFT_BINS))
#    frames = np.empty(dtype=DTYPE, shape=(max_samples,FFT_BINS,1))
#    targets = np.empty(dtype=DTYPE, shape=(max_samples,FFT_BINS))
    file_list = os.listdir(clean_dir)
    file_index = 0
    while (sample_index < max_samples) and (file_index < len(file_list)) :
        file = file_list[file_index]
        filename = os.fsdecode(file)
        noisy_ft = get_ft_from_file(noisy_dir + file)
        clean_ft = get_ft_from_file(clean_dir + file)
        diff = diff_ft(noisy_ft, clean_ft)
        if (diff > min_diff) and (diff < max_diff):
            new_frames = get_samples(noisy_dir + file)
            max_step = min(new_frames.shape[0], max_samples-sample_index)
            frames[sample_index:sample_index+max_step,:,0] = new_frames[:max_step,:]
            new_targets = get_samples(clean_dir + file)
            targets[sample_index:sample_index+max_step,:] = new_targets[:max_step,:]
            sample_index += new_targets.shape[0]
        file_index += 1
    print("Reached sample # " + str(min(sample_index, max_samples)))
    return frames, targets


In [36]:
# small test data "Assets\\DataShareArchive\\Test"
# 28K "F:\\Audiodata\\Train28Spk"
f, t = create_data("F:\\Audiodata\\Train28Spk", "f:\\Audiodata", max_samples=500000)

Reached sample # 500000
